In [ ]:
using MKL
using PhysicalUnits, LinearAlgebra
using MoireIVC.LLHF
using MoireIVC.LLHF: VP_solution
using MoireIVC.LLHF_Plot
using CairoMakie
CairoMakie.activate!()

In [ ]:
N1 = 18; N2 = 18; LL = 0
num_para = LLHF_init_with_lambda(1.0; N1 = N1, N2 = N2, LL = LL);
sym = [LLHF.Rot3(0);];
symT = [LLHF.Rot3(0); LLHF.PT(0,:T)];
symP = [LLHF.Rot3(0); LLHF.PT(0,:P)];
symPT = [LLHF.Rot3(0); LLHF.PT(0,:PT)];

In [ ]:
function find_crossing_lambda(sym, para = num_para)
    LLHF_change_lambda!(para, 0);
    EVP0 = LLHF_EnergyPerArea(VP_solution(para,2); para=para)
    ρ = LLHF_solve(para; coherence = 0.0, 
        error_tolerance = 1E-10, max_iter_times = 100, 
        post_process_times = 100, post_procession = sym,
        complusive_mixing=false, complusive_mixing_rate=0.5, 
        stepwise_output = false, final_output = false
    );
    EIVC0 = LLHF_EnergyPerArea(ρ; para=para)

    LLHF_change_lambda!(para, 1);
    EVP1 = LLHF_EnergyPerArea(VP_solution(para,2); para=para)
    ρ = LLHF_solve(para; coherence = 0.0, 
        error_tolerance = 1E-10, max_iter_times = 100, 
        post_process_times = 100, post_procession = sym,
        complusive_mixing=false, complusive_mixing_rate=0.5, 
        stepwise_output = false, final_output = false
    );
    EIVC1 = LLHF_EnergyPerArea(ρ; para=para)

    crossing = (EVP0-EIVC0) / (EIVC1-EVP1 + EVP0-EIVC0)
    @show crossing
    return crossing
end
function sweep_lambda(sym, lambda_sorted_list, input_i, input_dm;
    para = num_para, iter_times = 400, print=true)
    # 1-T 2-P 3-PT 4-IVC
    energies = similar(lambda_sorted_list, Float64)

    # input starting lambda
    LLHF_change_lambda!(para, lambda_sorted_list[input_i]);
    println("i = $input_i, λ = $(lambda_sorted_list[input_i])")
    ρ1 = LLHF_solve(para, input_dm; coherence = 0.0, 
        error_tolerance = 1E-10, max_iter_times = iter_times, 
        post_process_times = iter_times, post_procession = sym,
        complusive_mixing=false, complusive_mixing_rate=0.5, 
        stepwise_output = false, final_output = print
    );
    energies[input_i] = LLHF_EnergyPerArea(ρ1; para=para)
    ρ2 = copy(ρ1)


    for i in eachindex(lambda_sorted_list)[input_i:end]
        i == input_i && continue
        LLHF_change_lambda!(para, lambda_sorted_list[i]);
        println("i = $i, λ = $(lambda_sorted_list[i])");
        ρ1 = LLHF_solve(para, ρ1; coherence = 0.0, 
            error_tolerance = 1E-10, max_iter_times = iter_times, 
            post_process_times = iter_times, post_procession = sym,
            complusive_mixing=false, complusive_mixing_rate=0.5, 
            stepwise_output = false, final_output = print
        );
        energies[i] = LLHF_EnergyPerArea(ρ1; para=para)
    end
    for i in reverse(eachindex(lambda_sorted_list)[begin:input_i])
        i == input_i && continue
        LLHF_change_lambda!(para, lambda_sorted_list[i]);
        println("i = $i, λ = $(lambda_sorted_list[i])");
        ρ2 = LLHF_solve(para, ρ2; coherence = 0.0, 
            error_tolerance = 1E-10, max_iter_times = iter_times, 
            post_process_times = iter_times, post_procession = sym,
            complusive_mixing=false, complusive_mixing_rate=0.5, 
            stepwise_output = false, final_output = print
        );
        energies[i] = LLHF_EnergyPerArea(ρ2; para=para)
    end
    return energies
end
function sweep_lambda_fixρ(lambda_sorted_list, ρ; para=num_para)
    energies = similar(lambda_sorted_list, Float64)
    for i in eachindex(lambda_sorted_list)
        LLHF_change_lambda!(para, lambda_sorted_list[i]);
        energies[i] = LLHF_EnergyPerArea(ρ; para=para)
    end
    return energies
end
function solve(sym, times=300; para = num_para)
    LLHF_solve(para; coherence = 0.0, 
        error_tolerance = 1E-10, max_iter_times = times, 
        post_process_times = times, post_procession = sym,
        complusive_mixing=false, complusive_mixing_rate=0.5, 
        stepwise_output = false, final_output = true
    );
end
function solve!(ρ, sym, times=300; para = num_para)
    LLHF_solve(para, ρ; coherence = 0.0, 
        error_tolerance = 1E-10, max_iter_times = times, 
        post_process_times = times, post_procession = sym,
        complusive_mixing=false, complusive_mixing_rate=0.5, 
        stepwise_output = false, final_output = true
    );
end
@show aprx2W0 = round(0.2num_para.system.W0 / meV; digits=2);

#### H0=0

In [ ]:
lambda_plot0 = [0:0.05:0.4; 0.45:0.005:0.75; 0.8:0.1:1.2; 1.4:0.3:10.5]
LLHF.H0_C3_T!(num_para, 0.0);
crs = find_crossing_lambda(symPT)
iP = searchsortedfirst(lambda_plot0, crs)-1
LLHF_change_lambda!(num_para, lambda_plot0[iP]);
ρPT = solve(symPT);
ρP = solve(symP);
LLHF_plot_Sz(ρP; para=num_para);

iT = eachindex(lambda_plot0)[end]
LLHF_change_lambda!(num_para, lambda_plot0[iT]);
ρT = solve(symT);
LLHF_plot_Sz(ρT; para=num_para);


In [ ]:
@time begin
    energies0 = Matrix{Float64}(undef, 4, length(lambda_plot0));
    energies0[1,:] = sweep_lambda(symP, lambda_plot0, iP, copy(ρP); print = false);
    energies0[2,:] = sweep_lambda(symT, lambda_plot0, iT, copy(ρT); print = false);
    energies0[3,:] = sweep_lambda_fixρ(lambda_plot0, ρPT);
    energies0[4,:] = sweep_lambda_fixρ(lambda_plot0, VP_solution(num_para,2));
end
nothing

In [ ]:
@show xt_0 = lambda_plot0[findfirst(energies0[3,:] .- energies0[1,:] .> 1E-4)];
@show xp_0 = lambda_plot0[findlast(energies0[4,:] .- energies0[1,:] .> 1E-4)];

#### H0 - T

##### ho=0.1W0

In [ ]:
lambda_T = [0.57:0.005:0.70;]
LLHF.H0_C3_T!(num_para, 0.1);
crs = find_crossing_lambda(symT)
iP = searchsortedfirst(lambda_T, crs)-1
LLHF_change_lambda!(num_para, lambda_T[iP]);
ρP = solve!(ρP, symP);
ρP = solve!(ρP, sym);
LLHF_plot_Sz(ρP; para=num_para);

ρPT = solve(symPT);

iT = eachindex(lambda_T)[end]
LLHF_change_lambda!(num_para, lambda_T[iT]);
ρT = solve(symT);

In [ ]:
@time begin
    energiesT = Matrix{Float64}(undef, 4, length(lambda_T));
    energiesT[1,:] = sweep_lambda(sym , lambda_T, iP, copy(ρP); print = false);
    energiesT[2,:] = sweep_lambda(symT, lambda_T, iT, copy(ρT); print = false);
    imax = findmax(energiesT[1,:]-energiesT[2,:])[2] - 1
    energiesT[1, begin:imax] .= NaN
    energiesT[3,:] = sweep_lambda_fixρ(lambda_T, ρPT);
    energiesT[4,:] = sweep_lambda_fixρ(lambda_T, VP_solution(num_para,2));
end
nothing

In [ ]:
@show xt_t1 = lambda_T[findfirst(energiesT[2,:] .> energiesT[1,:])];
@show xp_t1 = lambda_T[findlast(energiesT[4,:] .- energiesT[1,:] .> 1E-4)];

##### ho=0.2W0

In [ ]:
lambda_plot_T = [0.57:0.005:0.73;]

In [ ]:
lambda_T = [0.57:0.005:0.73;]
lambda_plot_T = copy(lambda_T)
for x = 0.1:0.02:0.2
    LLHF.H0_C3_T!(num_para, x);
    crs = find_crossing_lambda(symT)
    iP = searchsortedfirst(lambda_T, crs)-1
    LLHF_change_lambda!(num_para, lambda_T[iP]);
    ρP = solve!(ρP, sym, 200);
end
ρP = solve!(ρP, sym);
LLHF_plot_Sz(ρP; para=num_para);

ρPT = solve(symPT);

iT = eachindex(lambda_T)[end]
LLHF_change_lambda!(num_para, lambda_T[iT]);
ρT = solve(symT);


In [ ]:
@time begin
    energiesT = Matrix{Float64}(undef, 4, length(lambda_T));
    energiesT[1,:] = sweep_lambda(sym , lambda_T, iP, copy(ρP); print = false);
    energiesT[2,:] = sweep_lambda(symT, lambda_T, iT, copy(ρT); print = false);
    imax = findmax(energiesT[1,:]-energiesT[2,:])[2] - 1
    energiesT[1, begin:imax] .= NaN
    energiesT[3,:] = sweep_lambda_fixρ(lambda_T, ρPT);
    energiesT[4,:] = sweep_lambda_fixρ(lambda_T, VP_solution(num_para,2));
end
energiesT_plot = copy(energiesT);

In [ ]:
@show xt_t2 = lambda_T[findfirst(energiesT[2,:] .> energiesT[1,:])];
@show xp_t2 = lambda_T[findlast(energiesT[4,:] .- energiesT[1,:] .> 1E-4)];

##### ho = 0.25W0

In [ ]:
lambda_T = [0.65:0.001:0.68;]
for x = 0.2:0.01:0.25
    LLHF.H0_C3_T!(num_para, x);
    crs = find_crossing_lambda(symT)
    iP = searchsortedfirst(lambda_T, crs)-1
    LLHF_change_lambda!(num_para, lambda_T[iP]);
    ρP = solve!(ρP, sym, 200);
end
ρP = solve!(ρP, sym);
LLHF_plot_Sz(ρP; para=num_para);

ρPT = solve(symPT);

iT = eachindex(lambda_T)[end]
LLHF_change_lambda!(num_para, lambda_T[iT]);
ρT = solve(symT);


In [ ]:
@time begin
    energiesT = Matrix{Float64}(undef, 4, length(lambda_T));
    energiesT[1,:] = sweep_lambda(sym , lambda_T, iP, copy(ρP); print = false);
    energiesT[2,:] = sweep_lambda(symT, lambda_T, iT, copy(ρT); print = false);
    imax = findmax(energiesT[1,:]-energiesT[2,:])[2] - 1
    energiesT[1, begin:imax] .= NaN
    energiesT[3,:] = sweep_lambda_fixρ(lambda_T, ρPT);
    energiesT[4,:] = sweep_lambda_fixρ(lambda_T, VP_solution(num_para,2));
end
nothing

In [ ]:
@show xt_t25 = lambda_T[findfirst(energiesT[2,:] .> energiesT[1,:])];
@show xp_t25 = lambda_T[findlast(energiesT[4,:] .- energiesT[1,:] .> 1E-4)];

##### h0 = 0.275W0, 1W0

In [ ]:
LLHF.H0_C3_T!(num_para, 0.275);
crst275 = find_crossing_lambda(symT);
LLHF.H0_C3_T!(num_para, 1);
crst1 = find_crossing_lambda(symT);

#### H0 - P side

##### he = 0.1 W0

In [ ]:
LLHF.H0_P!(num_para, 0.1);
lambda_P = [0.48:0.005:0.6;]
ip=1
LLHF_change_lambda!(num_para, lambda_P[iP]);
ρP = solve!(ρP, symP);
LLHF_plot_Sz(ρP; para=num_para);
ρPT = solve(symPT);

In [ ]:
@time begin
    energiesP = Matrix{Float64}(undef, 4, length(lambda_P));
    energiesP[1,:] = sweep_lambda(symP, lambda_P, iP, copy(ρP), print = false);
    energiesP[2,:] .= NaN
    energiesP[3,:] = sweep_lambda_fixρ(lambda_P, ρPT);
    energiesP[4,:] = sweep_lambda_fixρ(lambda_P, VP_solution(num_para,2));
end
nothing

In [ ]:
@show xp_p1 =lambda_P[findlast(energiesP[4,:] .- energiesP[1,:] .> 1E-4)];

##### he = 0.2W0

In [ ]:
lambda_P = [0:0.05:0.35; 0.4:0.005:0.5; 0.52:0.02:0.6]
lambda_plot_P = copy(lambda_P)
LLHF.H0_P!(num_para, 0.2);
crs = find_crossing_lambda(symPT)
iP = searchsortedfirst(lambda_P, crs)+2
LLHF_change_lambda!(num_para, lambda_P[iP]);
ρP = solve!(ρP, symP);
LLHF_plot_Sz(ρP; para=num_para);
ρPT = solve(symPT);

In [ ]:
@time begin
    energiesP = Matrix{Float64}(undef, 4, length(lambda_P));
    energiesP[1,:] = sweep_lambda(symP, lambda_P, iP, copy(ρP), print = false);
    energiesP[2,:] .= NaN
    energiesP[3,:] = sweep_lambda_fixρ(lambda_P, ρPT);
    energiesP[4,:] = sweep_lambda_fixρ(lambda_P, VP_solution(num_para,2));
end
energiesP_plot = copy(energiesP);

In [ ]:
@show xp_p2 = lambda_P[findlast(energiesP[4,:] .- energiesP[1,:] .> 1E-4)];

##### he = 0.3W0, 0.4W0, 0.5W0, 0.6W0

In [ ]:
LLHF.H0_P!(num_para, 0.3);
lambda_P = [0.25:0.01:0.45;]
iP = 8
LLHF_change_lambda!(num_para, lambda_P[iP]);
ρP = solve!(ρP, symP);
LLHF_plot_Sz(ρP; para=num_para);
energiesP = Matrix{Float64}(undef, 2, length(lambda_P));
energiesP[1,:] = sweep_lambda(symP, lambda_P, iP, copy(ρP), print = false);
energiesP[2,:] = sweep_lambda_fixρ(lambda_P, VP_solution(num_para,2));
@show xp_p3 = lambda_P[findlast(energiesP[2,:] .- energiesP[1,:] .> 1E-4)];


In [ ]:
LLHF.H0_P!(num_para, 0.4);
lambda_P = [0.2:0.01:0.35;]
iP = 5
LLHF_change_lambda!(num_para, lambda_P[iP]);
ρP = solve!(ρP, symP);
LLHF_plot_Sz(ρP; para=num_para);
energiesP = Matrix{Float64}(undef, 2, length(lambda_P));
energiesP[1,:] = sweep_lambda(symP, lambda_P, iP, copy(ρP), print = false);
energiesP[2,:] = sweep_lambda_fixρ(lambda_P, VP_solution(num_para,2));
@show xp_p4 = lambda_P[findlast(energiesP[2,:] .- energiesP[1,:] .> 1E-4)];

In [ ]:
LLHF.H0_P!(num_para, 0.5);
lambda_P = [0.1:0.01:0.25;]
iP = 5
LLHF_change_lambda!(num_para, lambda_P[iP]);
ρP = solve!(ρP, symP);
LLHF_plot_Sz(ρP; para=num_para);
energiesP = Matrix{Float64}(undef, 2, length(lambda_P));
energiesP[1,:] = sweep_lambda(symP, lambda_P, iP, copy(ρP), print = false);
energiesP[2,:] = sweep_lambda_fixρ(lambda_P, VP_solution(num_para,2));
@show xp_p5 = lambda_P[findlast(energiesP[2,:] .- energiesP[1,:] .> 1E-4)];

In [ ]:
LLHF.H0_P!(num_para, 0.6);
lambda_P = [0.0:0.01:0.2;]
iP = 7
LLHF_change_lambda!(num_para, lambda_P[iP]);
ρP = solve!(ρP, symP);
LLHF_plot_Sz(ρP; para=num_para);
energiesP = Matrix{Float64}(undef, 2, length(lambda_P));
energiesP[1,:] = sweep_lambda(symP, lambda_P, iP, copy(ρP), print = false);
energiesP[2,:] = sweep_lambda_fixρ(lambda_P, VP_solution(num_para,2));
@show xp_p6 = lambda_P[findlast(energiesP[2,:] .- energiesP[1,:] .> 1E-4)];

#### Plot

In [ ]:
eng_plot = Figure(size=(1400,700));
energylines = eng_plot[1,1] = GridLayout();
phasediagaram = eng_plot[1,2] = GridLayout();
colgap!(eng_plot.layout, 10);

In [ ]:
ax0 = Axis(energylines[1,1],
    subtitle = rich(
        "H", subscript("0"), " = 0",
    ),
    yticks = -1.0:0.5:0.0,
    yminorticks = [-0.75; -0.25],
    yminorgridvisible = true,
    yminorgridcolor = (:black, 0.12),
)
i2 = searchsortedfirst(lambda_plot0, 1.5)
lines!(lambda_plot0[1:i2], energies0[1,1:i2] ./ (meV/nm^2) )
lines!(lambda_plot0[1:i2], energies0[2,1:i2] ./ (meV/nm^2) )
lines!(lambda_plot0[1:i2], energies0[3,1:i2] ./ (meV/nm^2) )
lines!(lambda_plot0[1:i2], energies0[4,1:i2] ./ (meV/nm^2) )
ylims!(-1.05, 0.05)

ax0_T = Axis(energylines[1,1];
    width = Relative(0.4), height = Relative(0.3),
    halign = 0.25, valign = 0.2,
    limits = ((0.7,10.3),(-3.2,-0.5)),
    xticks = [1;5;10], xminorticks = [2:4;6:9], 
    xminorgridvisible = true, 
    xminorgridcolor = (:black, 0.12)
)
lines!(lambda_plot0, energies0[1,:] ./ (meV/nm^2) )
lines!(lambda_plot0, energies0[2,:] ./ (meV/nm^2) )
lines!(lambda_plot0, energies0[3,:] ./ (meV/nm^2) )
lines!(lambda_plot0, energies0[4,:] ./ (meV/nm^2) )
translate!(ax0_T.blockscene, 0, 0, 150)


ax0_P = Axis(energylines[2,1];
    subtitle = rich(
        "H", subscript("0"), " = 0",
    ),
    limits = ((0.515,0.68), (-0.39, -0.325)),
)
lines!(lambda_plot0, energies0[1,:] ./ (meV/nm^2), label="IVC-C1")
lines!(lambda_plot0, energies0[2,:] ./ (meV/nm^2), label="IVC-C0")
lines!(lambda_plot0, energies0[3,:] ./ (meV/nm^2), label="IVC-GL")
lines!(lambda_plot0, energies0[4,:] ./ (meV/nm^2), label="VP")
axislegend(position = :lb, )


ax1 = Axis(energylines[1,2],
    subtitle = rich(
        "h", subscript("o",fontsize = 12), " = 0.2W", subscript("0"), " = 16.15meV"
    ),
    yticks = -0.39:0.02:-0.35,
    yminorticks = [-0.38; -0.36],
    yminorgridvisible = true,
    yminorgridcolor = (:black, 0.12),
)
i2 = searchsortedfirst(lambda_plot_T,2)
lines!(lambda_plot_T, energiesT_plot[1,:] ./ (meV/nm^2), label="IVC-C1")
lines!(lambda_plot_T, energiesT_plot[2,:] ./ (meV/nm^2), label="IVC-C0")
lines!(lambda_plot_T, energiesT_plot[3,:] ./ (meV/nm^2), label="IVC-GL")
lines!(lambda_plot_T, energiesT_plot[4,:] ./ (meV/nm^2), label="VP")
xlims!(0.595, 0.682)
ylims!(-0.391, -0.348)

ax1_P = Axis(energylines[1,2];
    width = Relative(0.4), height = Relative(0.3),
    halign = 0.35, valign = 0.15,
    limits = ((0.628,0.642), (-0.371, -0.365)),
    xticks = 0.63:0.01:0.64,
    #yticks = -0.38:0.02:-0.33,
    xminorticks = [0.635], xminorgridcolor = (:black, 0.12),
    xminorgridvisible = true,
)
lines!(lambda_plot_T, energiesT_plot[1,:] ./ (meV/nm^2) )
lines!(lambda_plot_T, energiesT_plot[2,:] ./ (meV/nm^2) )
lines!(lambda_plot_T, energiesT_plot[3,:] ./ (meV/nm^2) )
lines!(lambda_plot_T, energiesT_plot[4,:] ./ (meV/nm^2) )
translate!(ax1_P.blockscene, 0, 0, 150)


ax2 = Axis(energylines[2,2],
    subtitle = rich(
        "h", subscript("e",fontsize = 12), " = 0.2W", subscript("0"), " = 16.15meV"
    ),
    yticks = -0.4:0.1:-0.2,
    yminorticks = [-0.25; -0.35],
    yminorgridvisible = true,
    yminorgridcolor = (:black, 0.12),
)
i2 = searchsortedfirst(lambda_plot_P,2)
lines!(lambda_plot_P, energiesP_plot[1,:] ./ (meV/nm^2), label="IVC-C1")
lines!(lambda_plot_P, energiesP_plot[2,:] ./ (meV/nm^2), label="IVC-C0")
lines!(lambda_plot_P, energiesP_plot[3,:] ./ (meV/nm^2), label="IVC-GL")
lines!(lambda_plot_P, energiesP_plot[4,:] ./ (meV/nm^2), label="VP")
xlims!(0.03, 0.55)
ylims!(-0.41, -0.17)


Label(energylines[3,1:2], fontsize = 19, text="λ")
Label(energylines[1:2,0], fontsize = 17, text="E/A (meV/nm²)", rotation = pi/2)
Label(energylines[1,1,TopLeft()], text = "(a)", halign=:left)
Label(energylines[2,1,TopLeft()], text = "(b)", halign=:left)
Label(energylines[1,2,TopLeft()], text = "(c)", halign=:left)
Label(energylines[2,2,TopLeft()], text = "(d)", halign=:left)

colgap!(energylines, 10) 

eng_plot

In [ ]:
ax_ph = Axis(phasediagaram[1,1];
    limits = ((-0.0, 1.0), (-0.525, 0.525)),
    width = 600, xticks = 0:0.25:1, 
    yticks = (-0.4:0.2:0.4, ["0.4"; "0.2"; "0.0"; "0.2"; "0.4"]),
    xgridvisible = false, ygridvisible = false,
)

lines!([xp_p6; xp_p4; xp_p2; xp_p1; xp_0; xp_t1; xp_t2; crst275], 
       [ -0.6;  -0.4;  -0.2;  -0.1;  0.0;   0.1;   0.2;   0.275];
    color = :black, linewidth = 2, linestyle = :dash
)
lines!([xt_0; 2.0;], [ 0.0;  0.0;];
    color = :black, linewidth = 2, linestyle = :dash
)
lines!([0.0; xt_0; xt_t1; xt_t2; crst275; crst1], 
       [0.0;  0.0;   0.1;   0.2;   0.275;   1.0];
    color = Makie.wong_colors()[4], linewidth = 2
)
poly!(Point2f[(xp_p6, -0.6), (xp_p4, -0.4), (xp_p2, -0.2), (xp_p1, -0.1), (xp_0, 0.0), 
    (xp_t1, 0.1), (xp_t2, 0.2), (crst275, 0.275), (crst1, 1.0), (1.5, 1.0), (1.5, -1.0)];
    color = (:grey, 0.2), strokewidth = 0
)
text!(0.72,  -0.3, text="P (C=1)", fontsize = 20, )
text!(0.15, -0.25, text="P (C=1)", fontsize = 20, )
text!(0.28,  0.25, text="T (C=0)", fontsize = 20, )
text!(0.2, -0.005, text="T+P (gapless)", fontsize = 20, )
text!(0.53,-0.005, text="P (C=1)", fontsize = 20, )
text!(0.77,-0.005, text="P (C=1)", fontsize = 20, )
text!(0.585, 0.07, text="(C=1)", fontsize = 20, )
text!(0.81,  0.22, text="(C=1)", fontsize = 20, )


Label(phasediagaram[2,1], text="λ", tellwidth = false, fontsize = 19)
Label(phasediagaram[1,1, Left()]; valign = 0.78, rotation = pi/2,
    text = rich("h", subscript("o",fontsize = 15), "( W", subscript("0"), ")"),
    padding = (0,30,0,0), fontsize = 17,
)
Label(phasediagaram[1,1, Left()]; valign = 0.22, rotation = pi/2,
    text = rich("h", subscript("e",fontsize = 15), "( W", subscript("0"), ")"),
    padding = (0,30,0,0), fontsize = 17,
)

Label(phasediagaram[1,1,TopLeft()], text = "(e)", halign=:left)

eng_plot

In [ ]:
save("phase_diagram.pdf", eng_plot, pt_per_unit = 1)